This notebook takes various data sources from the BLS and merges them into a master sheet that will be used in conjunction with the EEOC data to produce aggregate data.

## Load and merge data from BLS

The code below pulls data from these two Bureau of Labor Statistics datasets:

- Current Employment Statistics (for Sept. 2016, seasonally adjusted)
- The most recent National Occupational Employment and Wage Estimates

In [1]:
import pandas as pd

In [2]:
len(pd.read_csv('../data/CES_total_employment.csv'))

907

In [3]:
CES_total_employment = pd.read_csv('../data/CES_total_employment.csv')\
    .assign(metric="total_employment")

CES_avg_hrly_earnings = pd.read_csv('../data/CES_avg_hrly_earnings.csv')\
    .assign(metric="avg_hrly_earnings")
    
CES_women_employment = pd.read_csv('../data/CES_women_employment.csv')\
    .assign(metric="women_employment")

In [4]:
CES_total_employment.head()

,Series ID,Jan 2016,Feb 2016,Mar 2016,Apr 2016,May 2016,Jun 2016,Jul 2016,Aug 2016,Sep 2016,Oct 2016,Nov 2016,Dec 2016,metric
0,CES0000000001,143211.0,143448.0,143673.0,143826.0,143869.0,144166.0,144457.0,144633.0,144882.0,145006.0,145170.0,145325.0,total_employment
1,CES0500000001,121097.0,121318.0,121507.0,121665.0,121682.0,121951.0,122200.0,122343.0,122566.0,122698.0,122876.0,123026.0,total_employment
2,CES0600000001,19754.0,19747.0,19752.0,19743.0,19699.0,19705.0,19731.0,19707.0,19718.0,19727.0,19762.0,19794.0,total_employment
3,CES0700000001,123457.0,123701.0,123921.0,124083.0,124170.0,124461.0,124726.0,124926.0,125164.0,125279.0,125408.0,125531.0,total_employment
4,CES0800000001,101343.0,101571.0,101755.0,101922.0,101983.0,102246.0,102469.0,102636.0,102848.0,102971.0,103114.0,103232.0,total_employment


### Combine CES metrics

Note: See https://www.bls.gov/ces/cesnaics.htm#2.3 for details on Series ID structure.

In [5]:
ces_industry_metrics = pd.concat([
    CES_total_employment,
    CES_avg_hrly_earnings,
    CES_women_employment
])\
    .assign(industry_code=lambda x: x["Series ID"].str[3:11])\
    .set_index([ "industry_code",  "metric" ])["Sep 2016"].unstack()\
    .reset_index()\
    .assign(naics_supersector=lambda x: x["industry_code"].str[0:2])\
    .assign(naics_code=lambda x: x["industry_code"].str[2:8])\
    .assign(naics_sector_rollup=lambda x: x["naics_code"].str[:2])

ces_industry_metrics["women_percentage"] = ces_industry_metrics["women_employment"] / ces_industry_metrics["total_employment"]
ces_industry_metrics["source"] = "ces"
ces_industry_metrics.head()

metric,industry_code,avg_hrly_earnings,total_employment,women_employment,naics_supersector,naics_code,naics_sector_rollup,women_percentage,source
0,00000000,NaN,144882.0,71873.0,00,000000,00,0.496080,ces
1,05000000,25.81,122566.0,59073.0,05,000000,00,0.481969,ces
2,06000000,27.09,19718.0,4325.0,06,000000,00,0.219343,ces
3,07000000,NaN,125164.0,67548.0,07,000000,00,0.539676,ces
4,08000000,25.52,102848.0,54748.0,08,000000,00,0.532320,ces


In [6]:
len(ces_industry_metrics)

908

### Save industry metrics

In [7]:
ces_industry_metrics.to_csv('../output/ces_industry_metrics.csv', index=False)

## Sector metrics

### Load NAICS sectors

In [8]:
sectors = pd.read_csv(
    "../data/naics_sectors.csv",
    dtype={
        "naics_sector_rollup": str,
        "naics_supersector": str
    })
sectors

,naics_sector,naics_sector_rollup,naics_supersector,naics_sector_name
0,11,11,NaN,"Agriculture, Forestry, Fishing and Hunting"
1,21,21,10,"Mining, Quarrying, and Oil and Gas Extraction"
2,22,22,40,Utilities
3,23,23,20,Construction
4,31,31,30,Manufacturing
5,32,31,30,Manufacturing
6,33,31,30,Manufacturing
7,42,42,40,Wholesale Trade
8,44,44,40,Retail Trade
9,45,44,40,Retail Trade


### Extract the straightforward sector data from CES

I.e., CES series with the industry code "{NAICS_SECTOR_CODE}0000"

In [9]:
ces_sector_metrics = ces_industry_metrics[
    (ces_industry_metrics["naics_code"].str[2:] == "0000") &
    (ces_industry_metrics["naics_sector_rollup"] != "00")
]

ces_sector_metrics.head()

metric,industry_code,avg_hrly_earnings,total_employment,women_employment,naics_supersector,naics_code,naics_sector_rollup,women_percentage,source
7,10210000,32.95,610.2,92.6,10,210000,21,0.151754,ces
323,41420000,29.73,5877.5,1740.8,41,420000,42,0.296180,ces
516,44220000,38.35,555.9,129.4,44,220000,22,0.232776,ces
552,55520000,NaN,6167.0,NaN,55,520000,52,NaN,ces
593,55530000,NaN,2156.5,NaN,55,530000,53,NaN,ces


### Extract supersector data

This will be used, further down, to get female workforce numbers where not available in CES

In [10]:
ces_supersector_metrics = ces_industry_metrics[
    (ces_industry_metrics["naics_code"] == "000000")
]\
    .assign(naics_supersector=lambda x: x["industry_code"].str[:2])\
    [[ "naics_supersector", "avg_hrly_earnings", "total_employment", "women_percentage"]]\
    .assign(source="ces_supersector")\
    .pipe(pd.merge,
        sectors[[ "naics_sector_rollup", "naics_supersector" ]],
        how="left"
    )
ces_supersector_metrics.tail()

,naics_supersector,avg_hrly_earnings,total_employment,women_percentage,source,naics_sector_rollup
27,65,25.87,22745.0,0.770851,ces_supersector,62
28,70,15.01,15684.0,0.519319,ces_supersector,71
29,70,15.01,15684.0,0.519319,ces_supersector,72
30,80,23.17,5708.0,0.525578,ces_supersector,81
31,90,NaN,22316.0,0.573579,ces_supersector,92


### Manually add data for agriculture sector

For the agriculture sector, CES doesn't provide the number of women employees. Below, we add that data manually, based on the Current Population Survey:

In [11]:
ces_sector_metrics_manual = pd.DataFrame([
    {
        "naics_sector_rollup": "11",
        "women_percentage": 0.224,
        "source": "cps"
    } # Source: https://www.bls.gov/cps/cpsaat11.htm
])
ces_sector_metrics_manual

,naics_sector_rollup,source,women_percentage
0,11,cps,0.224


### Extract data from Occupational Employment Statistics

(For sectors without data in the CES.)

In [12]:
oes = pd.read_excel("../data/oes-natsector_M2016_dl.xlsx")
oes.head()

,NAICS,NAICS_TITLE,OCC_CODE,OCC_TITLE,OCC_GROUP,TOT_EMP,EMP_PRSE,PCT_TOTAL,PCT_RPT,H_MEAN,...,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
0,11,"Agriculture, Forestry, Fishing and Hunting",00-0000,Industry Total,total,416600,1.2,100,100,14.33,...,11.36,15.41,22.7,20040,20820,23630,32040,47210,NaN,NaN
1,11,"Agriculture, Forestry, Fishing and Hunting",11-0000,Management Occupations,major,7800,5.3,1.87,17,45.79,...,37.78,56.2,87.16,37490,53360,78580,116890,181290,NaN,NaN
2,11,"Agriculture, Forestry, Fishing and Hunting",11-1000,Top Executives,minor,3890,8.4,0.93,~,50.58,...,39.87,62.34,99.7,38140,55920,82930,129660,207380,NaN,NaN
3,11,"Agriculture, Forestry, Fishing and Hunting",11-1010,Chief Executives,broad,210,14.1,0.05,~,91.3,...,79.43,#,#,81690,98720,165210,#,#,NaN,NaN
4,11,"Agriculture, Forestry, Fishing and Hunting",11-1011,Chief Executives,detailed,210,14.1,0.05,1,91.3,...,79.43,#,#,81690,98720,165210,#,#,NaN,NaN


In [13]:
oes_sector_metrics = oes[
    oes["OCC_TITLE"] == "Industry Total"
].rename(columns={
    "TOT_EMP": "total_employment",
    "H_MEAN": "avg_hrly_earnings",
}).assign(naics_sector_rollup=lambda x: x["NAICS"].apply(lambda y: y.split("-")[0]))[[
    "naics_sector_rollup",
    "total_employment",
    "avg_hrly_earnings"
]]\
    .assign(total_employment=lambda x: x["total_employment"] / 1000)\
    .assign(naics_sector_rollup=lambda x: x["naics_sector_rollup"].replace("99", "92"))
oes_sector_metrics["source"] = "oes"
oes_sector_metrics

,naics_sector_rollup,total_employment,avg_hrly_earnings,source
0,11,416.6,14.33,oes
291,21,649.13,31.69,oes
843,22,549.96,36.35,oes
1372,23,6687.38,25.3,oes
2106,31,12337.5,24.39,oes
3101,42,5840.73,27.01,oes
4023,44,15982.5,15.44,oes
4839,48,5606.18,22.57,oes
5658,51,2762.09,34.86,oes
6285,52,5775.24,33.5,oes


### Combine all metrics, in order of preference

In [14]:
sector_metrics_tidy = pd.concat([
    ces_sector_metrics.assign(priority=1),
    ces_sector_metrics_manual.assign(priority=2),
    oes_sector_metrics.assign(priority=3),
    ces_supersector_metrics.assign(priority=4),
]).pipe(pd.melt,
        id_vars=["naics_sector_rollup", "priority", "source"]
    ).sort_values("priority")\
    .pipe(lambda x: x[x["variable"].isin([ "avg_hrly_earnings", "total_employment", "women_percentage" ])])\
    .dropna(subset=["value"])

sector_metrics_tidy.head()

,naics_sector_rollup,priority,source,variable,value
0,21,1,ces,avg_hrly_earnings,32.95
285,92,1,ces,total_employment,5101
284,91,1,ces,total_employment,2811
283,72,1,ces,total_employment,13449.2
282,71,1,ces,total_employment,2235.2


In [15]:
sector_metrics_tidy_unique = sector_metrics_tidy.groupby([
    "naics_sector_rollup",
    "variable"
]).first().reset_index()
sector_metrics_tidy_unique.head()

,naics_sector_rollup,variable,priority,source,value
0,11,avg_hrly_earnings,3,oes,14.330
1,11,total_employment,3,oes,416.600
2,11,women_percentage,2,cps,0.224
3,21,avg_hrly_earnings,1,ces,32.950
4,21,total_employment,1,ces,610.200


Here are the data sources of each variable we're ultimately using:

In [16]:
sector_metrics_tidy_unique.groupby([
    "source",
    "variable"
]).size().unstack().fillna(0).astype(int)\

variable,avg_hrly_earnings,total_employment,women_percentage
source,,,
ces,9,15,13
ces_supersector,0,0,8
cps,0,0,1
oes,11,7,0


In [17]:
sector_metrics_wide = sector_metrics_tidy_unique.set_index([
    "naics_sector_rollup",
    "variable"
])["value"].unstack().reset_index()

In [18]:
sector_metrics = pd.merge(
    sectors,
    sector_metrics_wide[[
        "naics_sector_rollup",
        "avg_hrly_earnings",
        "total_employment",
        "women_percentage"
    ]],
    how="left",
    on=["naics_sector_rollup"]
).pipe(pd.merge,
    sector_metrics_tidy_unique.groupby("naics_sector_rollup")["source"].unique()\
        .apply(",".join).reset_index(),
    on="naics_sector_rollup",
    how="left"
)

sector_metrics

,naics_sector,naics_sector_rollup,naics_supersector,naics_sector_name,avg_hrly_earnings,total_employment,women_percentage,source
0,11,11,NaN,"Agriculture, Forestry, Fishing and Hunting",14.33,416.60,0.224000,"oes,cps"
1,21,21,10,"Mining, Quarrying, and Oil and Gas Extraction",32.95,610.20,0.151754,ces
2,22,22,40,Utilities,38.35,555.90,0.232776,ces
3,23,23,20,Construction,25.30,6687.38,0.125019,"oes,ces_supersector"
4,31,31,30,Manufacturing,24.39,12337.52,0.274534,"oes,ces_supersector"
5,32,31,30,Manufacturing,24.39,12337.52,0.274534,"oes,ces_supersector"
6,33,31,30,Manufacturing,24.39,12337.52,0.274534,"oes,ces_supersector"
7,42,42,40,Wholesale Trade,29.73,5877.50,0.296180,ces
8,44,44,40,Retail Trade,15.44,15982.52,0.406632,"oes,ces_supersector"
9,45,44,40,Retail Trade,15.44,15982.52,0.406632,"oes,ces_supersector"


## Save combined data

In [19]:
sector_metrics.to_csv('../output/bls_sector_metrics.csv', index=False)

## Merge in custom information for graphics

In [20]:
graphics_info = pd.read_csv(
    '../data/graphics_info.csv',
    dtype={
        "naics_code": str,
        "focus" :str,
        "index_num":str
    }
)

graphics_info.head()

,industry,naics_code,grouping,industry_class,grouping_class,color,focus,index_num,info_text
0,Accommodation and Food Services,72,Service and sales-related jobs,accommodationandfoodservices,serviceandsalesrelatedjobs,#af2469,1,1,"By far, the most claims were filed by service-..."
1,Retail Trade,44,Service and sales-related jobs,retailtrade,serviceandsalesrelatedjobs,#f43192,1,2,"By far, the most claims were filed by service-..."
2,Other Services (except Public Administration),81,Service and sales-related jobs,otherservices,serviceandsalesrelatedjobs,#efb4d1,1,3,"By far, the most claims were filed by service-..."
3,Wholesale Trade,42,Service and sales-related jobs,wholesaletrade,serviceandsalesrelatedjobs,#efdce7,1,4,"By far, the most claims were filed by service-..."
4,Manufacturing,31,Manual Labor,manufacturing,manuallabor,#096c5f,2,1,"Manual labor jobs like construction, warehousi..."


In [21]:
len(graphics_info)

21

The next file includes information necessary for the graphic:

In [22]:
graphics_info_bls = pd.merge(
    graphics_info,
    sector_metrics.drop_duplicates(subset=["naics_sector_name"]),
    left_on="industry",
    right_on="naics_sector_name",
    how="left"
)

graphics_info_bls.head()

,industry,naics_code,grouping,industry_class,grouping_class,color,focus,index_num,info_text,naics_sector,naics_sector_rollup,naics_supersector,naics_sector_name,avg_hrly_earnings,total_employment,women_percentage,source
0,Accommodation and Food Services,72,Service and sales-related jobs,accommodationandfoodservices,serviceandsalesrelatedjobs,#af2469,1,1,"By far, the most claims were filed by service-...",72.0,72,70,Accommodation and Food Services,14.10,13449.20,0.527273,ces
1,Retail Trade,44,Service and sales-related jobs,retailtrade,serviceandsalesrelatedjobs,#f43192,1,2,"By far, the most claims were filed by service-...",44.0,44,40,Retail Trade,15.44,15982.52,0.406632,"oes,ces_supersector"
2,Other Services (except Public Administration),81,Service and sales-related jobs,otherservices,serviceandsalesrelatedjobs,#efb4d1,1,3,"By far, the most claims were filed by service-...",81.0,81,80,Other Services (except Public Administration),19.41,4078.80,0.525578,"oes,ces_supersector"
3,Wholesale Trade,42,Service and sales-related jobs,wholesaletrade,serviceandsalesrelatedjobs,#efdce7,1,4,"By far, the most claims were filed by service-...",42.0,42,40,Wholesale Trade,29.73,5877.50,0.296180,ces
4,Manufacturing,31,Manual Labor,manufacturing,manuallabor,#096c5f,2,1,"Manual labor jobs like construction, warehousi...",31.0,31,30,Manufacturing,24.39,12337.52,0.274534,"oes,ces_supersector"


In [23]:
len(graphics_info_bls)

21

In [24]:
graphics_info_bls.to_csv(
    "../output/graphics_info_bls.csv",
    index=False
)

---

---

---